In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.decomposition import FactorAnalysis

In [26]:
df_raw = pd.read_csv("features_raw.csv")
df_raw

,RealTime,TimeStamp,head0_head0_trans_x,head0_head0_trans_y,head0_head0_trans_z,head0_head0_ori_x,head0_head0_ori_y,head0_head0_ori_z,head0_head0_ori_w,head0_head1_trans_x,...,head2_AU04,head2_AU12,head2_AU14,head2_AU15,head0_pose,head1_pose,head2_pose,Class_P0,ClassP1,ClassP2
0,00:01,1597420660,0,0,0,0,0,0,1,0.437456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1,00:03,1597420686,0,0,0,0,0,0,1,-0.918670,...,0.0,0.0,1.0,1,armsCrossed,handsOnFace,handsOnFace,1.0,1.0,1.0
2,00:04,1597420687,0,0,0,0,0,0,1,-0.908760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3,00:04,1597420687,0,0,0,0,0,0,1,-0.908760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
4,00:06,1597420688,0,0,0,0,0,0,1,-0.332478,...,0.0,0.0,0.0,0,armsCrossed,handsOnFace,handsOnFace,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,11:27,1597421256,0,0,0,0,0,0,1,-0.401989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,11:28,1597421257,0,0,0,0,0,0,1,-0.401989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,11:30,1597421258,0,0,0,0,0,0,1,-0.401989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502,11:32,1597421260,0,0,0,0,0,0,1,-0.690607,...,0.0,1.0,1.0,1,other,other,other,NaN,NaN,NaN


In [59]:
cols = ["ts", "A_tx", "A_ty", "A_tz", "A_rx", "A_ry", "A_rz", "A_rw", "B_tx", "B_ty", "B_tz", "B_rx", "B_ry", "B_rz", "B_rw", "A_AU28", "A_AU45", "A_AU26", "A_AU25", "A_AU01", "A_AU05", "A_AU20", "A_AU06", "A_AU23", "A_AU07", "A_AU02", "A_AU09", "A_AU17", "A_AU10", "A_AU04", "A_AU12", "A_AU14", "A_AU15", "B_AU28", "B_AU45", "B_AU26", "B_AU25", "B_AU01", "B_AU05", "B_AU20", "B_AU06", "B_AU23", "B_AU07", "B_AU02", "B_AU09", "B_AU17", "B_AU10", "B_AU04", "B_AU12", "B_AU14", "B_AU15", "A_emo", "B_emo", "A_pose", "B_pose", "A_interactive", "B_interactive", "target_interactive"]
df0 = df_raw[["TimeStamp", "head0_head1_trans_x", "head0_head1_trans_y", "head0_head1_trans_z", "head0_head1_ori_x", "head0_head1_ori_y", "head0_head1_ori_z", "head0_head1_ori_w", "head0_head2_trans_x", "head0_head2_trans_y", "head0_head2_trans_z", "head0_head2_ori_x", "head0_head2_ori_y", "head0_head2_ori_z", "head0_head2_ori_w", "head1_AU28", "head1_AU45", "head1_AU26", "head1_AU25", "head1_AU01", "head1_AU05", "head1_AU20", "head1_AU06", "head1_AU23", "head1_AU07", "head1_AU02", "head1_AU09", "head1_AU17", "head1_AU10", "head1_AU04", "head1_AU12", "head1_AU14", "head1_AU15", "head2_AU28", "head2_AU45", "head2_AU26", "head2_AU25", "head2_AU01", "head2_AU05", "head2_AU20", "head2_AU06", "head2_AU23", "head2_AU07", "head2_AU02", "head2_AU09", "head2_AU17", "head2_AU10", "head2_AU04", "head2_AU12", "head2_AU14", "head2_AU15", "head1_emo", "head2_emo", "head1_pose", "head2_pose", "ClassP1", "ClassP2", "Class_P0"]]
df0.columns = cols

df1 = df_raw[["TimeStamp", "head1_head0_trans_x", "head1_head0_trans_y", "head1_head0_trans_z", "head1_head0_ori_x", "head1_head0_ori_y", "head1_head0_ori_z", "head1_head0_ori_w", "head1_head2_trans_x", "head1_head2_trans_y", "head1_head2_trans_z", "head1_head2_ori_x", "head1_head2_ori_y", "head1_head2_ori_z", "head1_head2_ori_w", "head0_AU28", "head0_AU45", "head0_AU26", "head0_AU25", "head0_AU01", "head0_AU05", "head0_AU20", "head0_AU06", "head0_AU23", "head0_AU07", "head0_AU02", "head0_AU09", "head0_AU17", "head0_AU10", "head0_AU04", "head0_AU12", "head0_AU14", "head0_AU15", "head2_AU28", "head2_AU45", "head2_AU26", "head2_AU25", "head2_AU01", "head2_AU05", "head2_AU20", "head2_AU06", "head2_AU23", "head2_AU07", "head2_AU02", "head2_AU09", "head2_AU17", "head2_AU10", "head2_AU04", "head2_AU12", "head2_AU14", "head2_AU15", "head0_emo", "head2_emo", "head0_pose", "head2_pose", "Class_P0", "ClassP2", "ClassP1"]]
df1.columns = cols

df2 = df_raw[["TimeStamp", "head2_head0_trans_x", "head2_head0_trans_y", "head2_head0_trans_z", "head2_head0_ori_x", "head2_head0_ori_y", "head2_head0_ori_z", "head2_head0_ori_w", "head2_head1_trans_x", "head2_head1_trans_y", "head2_head1_trans_z", "head2_head1_ori_x", "head2_head1_ori_y", "head2_head1_ori_z", "head2_head1_ori_w", "head0_AU28", "head0_AU45", "head0_AU26", "head0_AU25", "head0_AU01", "head0_AU05", "head0_AU20", "head0_AU06", "head0_AU23", "head0_AU07", "head0_AU02", "head0_AU09", "head0_AU17", "head0_AU10", "head0_AU04", "head0_AU12", "head0_AU14", "head0_AU15", "head1_AU28", "head1_AU45", "head1_AU26", "head1_AU25", "head1_AU01", "head1_AU05", "head1_AU20", "head1_AU06", "head1_AU23", "head1_AU07", "head1_AU02", "head1_AU09", "head1_AU17", "head1_AU10", "head1_AU04", "head1_AU12", "head1_AU14", "head1_AU15", "head0_emo", "head1_emo", "head0_pose", "head1_pose", "Class_P0", "ClassP1", "ClassP2"]]
df2.columns = cols

df = pd.concat([df0, df1, df2])
df = df.dropna()
df

,ts,A_tx,A_ty,A_tz,A_rx,A_ry,A_rz,A_rw,B_tx,B_ty,...,B_AU12,B_AU14,B_AU15,A_emo,B_emo,A_pose,B_pose,A_interactive,B_interactive,target_interactive
1,1597420686,-0.918670,0.200940,0.528746,-0.315028,0.202272,0.024085,0.926965,-0.403263,0.100988,...,0.0,1.0,1,anger,supprise,handsOnFace,handsOnFace,1.0,1.0,1.0
4,1597420688,-0.332478,0.108224,0.345611,-0.129217,0.039880,-0.102003,0.985550,-1.061279,0.260625,...,0.0,0.0,0,neutual,supprise,handsOnFace,handsOnFace,1.0,1.0,1.0
7,1597420690,0.968457,-0.153347,0.169623,-0.037320,-0.246060,-0.074364,0.965677,0.642247,-0.055239,...,0.0,0.0,0,anger,supprise,handsOnFace,armsCrossed,1.0,1.0,1.0
8,1597420693,1.065348,-0.034013,0.270835,-0.034176,-0.495714,0.042668,0.866764,0.692682,-0.005259,...,0.0,0.0,0,neutual,supprise,handsOnFace,handsOnFace,1.0,0.0,1.0
9,1597420694,0.963564,-0.072421,0.173688,-0.066512,-0.405960,-0.089201,0.907092,0.621916,-0.012438,...,0.0,1.0,0,happy,anger,handsOnFace,handsOnFace,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1597421144,-0.686312,0.297115,0.372977,0.040040,0.259542,-0.203127,0.943279,-1.310243,0.368795,...,0.0,0.0,1,neutual,happy,other,other,0.0,1.0,0.0
375,1597421145,-0.715979,0.354469,0.320770,0.081419,0.169535,-0.239006,0.952631,-1.322474,0.458673,...,0.0,0.0,1,happy,neutual,other,other,1.0,1.0,0.0
381,1597421148,-0.707426,-0.038731,-0.255703,-0.017435,0.013188,0.040374,0.998945,0.702203,-0.153591,...,1.0,0.0,1,happy,neutual,error,other,0.0,1.0,0.0
382,1597421151,0.456976,0.126349,0.513736,-0.114515,-0.404644,0.065291,0.904924,-0.527159,-0.281273,...,1.0,1.0,1,happy,neutual,other,error,0.0,1.0,0.0



# Person 0

## Preprocessing

In [58]:
df = pd.read_csv("/home/joe/MLP/features_sample_head0_selection.csv")
df = df.astype({'Class_P0': 'bool'})
#                 'head1_AU28': 'bool',
#                 'head1_AU45': 'bool',
#                 'head1_AU26': 'bool',
#                 'head1_AU25': 'bool',
#                 'head1_AU01': 'bool',
#                 'head1_AU05': 'bool',
#                 'head1_AU20': 'bool',
#                 'head1_AU06': 'bool',
#                 'head1_AU23': 'bool',
#                 'head1_AU07': 'bool',
#                 'head1_AU02': 'bool', 
#                 'head1_AU09': 'bool',
#                 'head1_AU17': 'bool',
#                 'head1_AU10': 'bool',
#                 'head1_AU04': 'bool',
#                 'head1_AU12': 'bool',
#                 'head1_AU14': 'bool',
#                 'head1_AU15': 'bool'})
# le = LabelEncoder()
# df['head0_pose'] = le.fit_transform(df['head0_pose'])
# df['head1_pose'] = le.fit_transform(df['head1_pose'])
# df['head2_pose'] = le.fit_transform(df['head2_pose'])
# df['head0_emo'] = le.fit_transform(df['head0_emo'])
# df['head1_emo'] = le.fit_transform(df['head1_emo'])
# df['head2_emo'] = le.fit_transform(df['head2_emo'])

y = df['Class_P0']
x = df.drop(['Class_P0'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.30, random_state=27)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp = imp.fit(x_train)

X_train_imp = imp.transform(x_train)
X_test_imp = imp.transform(x_test)

FileNotFoundError: [Errno 2] File /home/joe/MLP/features_sample_head0_selection.csv does not exist: '/home/joe/MLP/features_sample_head0_selection.csv'

In [ ]:
df = df.astype({'target_interactive': 'bool',
                 'A_interactive': 'bool',
                'B_interactive': 'bool',
                 'A_AU28': 'bool',
                 'A_AU45': 'bool',
                 'A_AU26': 'bool',
                 'A_AU25': 'bool',
                 'A_AU01': 'bool',
                 'A_AU05': 'bool',
                 'A_AU20': 'bool',
                 'A_AU06': 'bool',
                 'A_AU23': 'bool',
                 'A_AU07': 'bool',
                 'A_AU02': 'bool', 
                 'A_AU09': 'bool',
                 'A_AU17': 'bool',
                 'A_AU10': 'bool',
                 'A_AU04': 'bool',
                 'A_AU12': 'bool',
                 'A_AU14': 'bool',
                 'A_AU15': 'bool',             
                 'B_AU28': 'bool',
                 'B_AU45': 'bool',
                 'B_AU26': 'bool',
                 'B_AU25': 'bool',
                 'B_AU01': 'bool',
                 'B_AU05': 'bool',
                 'B_AU20': 'bool',
                 'B_AU06': 'bool',
                 'B_AU23': 'bool',
                 'B_AU07': 'bool',
                 'B_AU02': 'bool', 
                 'B_AU09': 'bool',
                 'B_AU17': 'bool',
                 'B_AU10': 'bool',
                 'B_AU04': 'bool',
                 'B_AU12': 'bool',
                 'B_AU14': 'bool',
                 'B_AU15': 'bool'})


In [60]:

le = LabelEncoder()
poses_labels = le.fit(df["A_pose"])
df['A_pose'] = poses_labels.transform(df['A_pose'])
df['B_pose'] = poses_labels.transform(df['B_pose'])

emo_labels = le.fit(df["A_emo"])
df['A_emo'] = emo_labels.transform(df['A_emo'])
df['B_emo'] = emo_labels.transform(df['B_emo'])


y = df['target_interactive']
x = df.drop(['target_interactive'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.30, random_state=27)

df.dtypes


ts                      int64
A_tx                  float64
A_ty                  float64
A_tz                  float64
A_rx                  float64
A_ry                  float64
A_rz                  float64
A_rw                  float64
B_tx                  float64
B_ty                  float64
B_tz                  float64
B_rx                  float64
B_ry                  float64
B_rz                  float64
B_rw                  float64
A_AU28                 object
A_AU45                 object
A_AU26                 object
A_AU25                float64
A_AU01                float64
A_AU05                float64
A_AU20                float64
A_AU06                float64
A_AU23                float64
A_AU07                float64
A_AU02                float64
A_AU09                float64
A_AU17                float64
A_AU10                float64
A_AU04                float64
A_AU12                float64
A_AU14                float64
A_AU15                 object
B_AU28    

In [61]:
y

1      1.0
4      1.0
7      1.0
8      1.0
9      1.0
      ... 
374    0.0
375    0.0
381    0.0
382    0.0
383    0.0
Name: target_interactive, Length: 734, dtype: float64

In [62]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), verbose=10)

In [63]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)


Iteration 1, loss = inf
Iteration 2, loss = inf
Iteration 3, loss = inf
Iteration 4, loss = inf
Iteration 5, loss = inf
Iteration 6, loss = inf
Iteration 7, loss = inf
Iteration 8, loss = inf
Iteration 9, loss = inf
Iteration 10, loss = inf
Iteration 11, loss = inf
Iteration 12, loss = inf
Iteration 13, loss = inf
Iteration 14, loss = inf
Iteration 15, loss = inf
Iteration 16, loss = inf
Iteration 17, loss = inf
Iteration 18, loss = inf
Iteration 19, loss = inf
Iteration 20, loss = inf
Iteration 21, loss = inf
Iteration 22, loss = inf
Iteration 23, loss = inf
Iteration 24, loss = inf
Iteration 25, loss = inf
Iteration 26, loss = inf
Iteration 27, loss = inf
Iteration 28, loss = inf
Iteration 29, loss = inf
Iteration 30, loss = inf
Iteration 31, loss = inf
Iteration 32, loss = inf
Iteration 33, loss = inf
Iteration 34, loss = inf
Iteration 35, loss = inf
Iteration 36, loss = inf
Iteration 37, loss = inf
Iteration 38, loss = inf
Iteration 39, loss = inf
Iteration 40, loss = inf
Iteration

/usr/lib/python3/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [49]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.42081447963800905
[[  0 128]
 [  0  93]]
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       128
        True       0.42      1.00      0.59        93

    accuracy                           0.42       221
   macro avg       0.21      0.50      0.30       221
weighted avg       0.18      0.42      0.25       221



/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Feature Selection (Skip as it Takes time) 

In [5]:
# sfsl = sfs(clf, k_features=10,forward=True)
# print(sfsl.fit(X_test_imp, y_test))
# print(sfsl.subsets_)
# #'feature_names': head1_head0_trans_x	head1_head0_trans_y	head1_head0_ori_w	head1_head1_trans_x	head1_head1_ori_x	head1_head1_ori_y	head1_head2_trans_z	head1_AU07	head1_AU09	head1_AU12	ClassP1	Class_P0


## Factore analysis

In [6]:
factore = FactorAnalysis().fit(X_train_imp)
pd.DataFrame(factore.components_)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.388506,0.087763,0.005932,-0.396647,-0.015126,0.117161,-0.051744,-0.123371,0.011002,0.007325,-0.071395
1,-0.056144,-0.071599,-0.029599,-0.057255,0.039622,0.014710,0.005869,-0.109926,-0.040776,-0.032834,-0.066033
2,-0.067947,0.011947,0.000442,0.048567,0.009669,-0.013527,-0.052712,-0.017093,0.006009,-0.071062,0.043702
3,-0.029889,-0.004201,-0.001778,-0.006503,0.006608,-0.003135,-0.011832,0.056646,0.055773,0.013911,-0.050988
4,-0.025074,0.000669,-0.001575,0.003273,0.002489,-0.000907,0.015107,-0.020778,0.024504,0.030926,0.053669
5,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000
6,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000
8,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
9,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000


## Create the model 

In [7]:
clf.fit(X_train_imp, y_train)
y_pred = clf.predict(X_test_imp)



Iteration 1, loss = 0.62267323
Iteration 2, loss = 0.59164631
Iteration 3, loss = 0.57340501
Iteration 4, loss = 0.56795669
Iteration 5, loss = 0.56122511
Iteration 6, loss = 0.55591731
Iteration 7, loss = 0.55054527
Iteration 8, loss = 0.54576695
Iteration 9, loss = 0.54280397
Iteration 10, loss = 0.53962543
Iteration 11, loss = 0.53612381
Iteration 12, loss = 0.53291764
Iteration 13, loss = 0.53041010
Iteration 14, loss = 0.52699700
Iteration 15, loss = 0.52065848
Iteration 16, loss = 0.51574210
Iteration 17, loss = 0.50865140
Iteration 18, loss = 0.50109449
Iteration 19, loss = 0.49262554
Iteration 20, loss = 0.48243906
Iteration 21, loss = 0.47315107
Iteration 22, loss = 0.46109615
Iteration 23, loss = 0.44825852
Iteration 24, loss = 0.43447873
Iteration 25, loss = 0.42074495
Iteration 26, loss = 0.40843114
Iteration 27, loss = 0.39302677
Iteration 28, loss = 0.38030792
Iteration 29, loss = 0.36867777
Iteration 30, loss = 0.35617889
Iteration 31, loss = 0.34479597
Iteration 32, los

Iteration 255, loss = 0.18708513
Iteration 256, loss = 0.18658133
Iteration 257, loss = 0.18922548
Iteration 258, loss = 0.18883779
Iteration 259, loss = 0.18781648
Iteration 260, loss = 0.18675342
Iteration 261, loss = 0.18565299
Iteration 262, loss = 0.18533921
Iteration 263, loss = 0.18447226
Iteration 264, loss = 0.18445728
Iteration 265, loss = 0.18415260
Iteration 266, loss = 0.18385089
Iteration 267, loss = 0.18351705
Iteration 268, loss = 0.18322352
Iteration 269, loss = 0.18214653
Iteration 270, loss = 0.18300482
Iteration 271, loss = 0.18363204
Iteration 272, loss = 0.18192743
Iteration 273, loss = 0.18193245
Iteration 274, loss = 0.18092115
Iteration 275, loss = 0.18236923
Iteration 276, loss = 0.18116372
Iteration 277, loss = 0.17891853
Iteration 278, loss = 0.18016102
Iteration 279, loss = 0.17843205
Iteration 280, loss = 0.17851858
Iteration 281, loss = 0.17845584
Iteration 282, loss = 0.17815492
Iteration 283, loss = 0.17775145
Iteration 284, loss = 0.17641057
Iteration 

/home/joe/.local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Scores 

In [8]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8425531914893617
[[ 57  21]
 [ 16 141]]
              precision    recall  f1-score   support

       False       0.78      0.73      0.75        78
        True       0.87      0.90      0.88       157

   micro avg       0.84      0.84      0.84       235
   macro avg       0.83      0.81      0.82       235
weighted avg       0.84      0.84      0.84       235



# Person 1

In [13]:
df = pd.read_csv("/home/joe/MLP/features_sample_head1_selection.csv")
df = df.astype({'Class_P0': 'bool', 
#                 'head0_AU28': 'bool',
#                 'head0_AU45': 'bool',
#                 'head0_AU26': 'bool',
#                 'head0_AU25': 'bool',
#                 'head0_AU01': 'bool',
#                 'head0_AU05': 'bool',
#                 'head0_AU20': 'bool',
#                 'head0_AU06': 'bool',
#                 'head0_AU23': 'bool',
#                 'head0_AU07': 'bool',
#                 'head0_AU02': 'bool', 
#                 'head0_AU09': 'bool',
#                 'head0_AU17': 'bool',
#                 'head0_AU10': 'bool',
#                 'head0_AU04': 'bool',
#                 'head0_AU12': 'bool',
#                 'head0_AU14': 'bool',
#                 'head0_AU15': 'bool',
                'ClassP1':'bool'})

le = LabelEncoder()
# df['head0_pose'] = le.fit_transform(df['head0_pose'])
# df['head0_pose'] = le.fit_transform(df['head0_pose'])
# df['head2_pose'] = le.fit_transform(df['head2_pose'])
# df['head0_emo'] = le.fit_transform(df['head0_emo'])
# df['head0_emo'] = le.fit_transform(df['head0_emo'])
# df['head2_emo'] = le.fit_transform(df['head2_emo'])

y = df['ClassP1']
x = df.drop(['ClassP1'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.30, random_state=27)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp = imp.fit(x_train)

X_train_imp = imp.transform(x_train)
X_test_imp = imp.transform(x_test)

In [23]:
df.dtypes

head0_head0_trans_x    float64
head0_head0_trans_y    float64
head0_head0_ori_y      float64
head0_head0_ori_w      float64
head0_head1_trans_x    float64
head0_head1_trans_z    float64
head0_head1_ori_y      float64
head0_head2_trans_x    float64
head0_head2_ori_x      float64
head0_AU26             float64
Class_P0                  bool
ClassP2                   bool
dtype: object

In [14]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=300, alpha=0.0001, activation='tanh',
                     solver='adam', verbose=10,  random_state=21,tol=0.000000001)

In [15]:
# sfsl = sfs(clf, k_features=10,forward=True)
# print(sfsl.fit(X_test_imp, y_test))
# print(sfsl.subsets_)
# ('0', '7', '14', '16', '18', '20', '22', '27', '28', '32')

In [16]:
clf.fit(X_train_imp, y_train)
y_pred = clf.predict(X_test_imp)

Iteration 1, loss = 0.65825207
Iteration 2, loss = 0.53385928
Iteration 3, loss = 0.54384695
Iteration 4, loss = 0.52560267
Iteration 5, loss = 0.49881361
Iteration 6, loss = 0.49570625
Iteration 7, loss = 0.49703455
Iteration 8, loss = 0.48516131
Iteration 9, loss = 0.47805795
Iteration 10, loss = 0.47698585
Iteration 11, loss = 0.47402489
Iteration 12, loss = 0.46805799
Iteration 13, loss = 0.46642760
Iteration 14, loss = 0.46523517
Iteration 15, loss = 0.46249839
Iteration 16, loss = 0.46047774
Iteration 17, loss = 0.45990180
Iteration 18, loss = 0.45779835
Iteration 19, loss = 0.45630455
Iteration 20, loss = 0.45400427
Iteration 21, loss = 0.45082466
Iteration 22, loss = 0.44902884
Iteration 23, loss = 0.44553636
Iteration 24, loss = 0.44103916
Iteration 25, loss = 0.43686816
Iteration 26, loss = 0.43298512
Iteration 27, loss = 0.42830069
Iteration 28, loss = 0.42214901
Iteration 29, loss = 0.41564254
Iteration 30, loss = 0.40896665
Iteration 31, loss = 0.39995987
Iteration 32, los

Iteration 256, loss = 0.05089990
Iteration 257, loss = 0.04820587
Iteration 258, loss = 0.04978308
Iteration 259, loss = 0.04720463
Iteration 260, loss = 0.04744498
Iteration 261, loss = 0.04724125
Iteration 262, loss = 0.04682765
Iteration 263, loss = 0.04555938
Iteration 264, loss = 0.04667503
Iteration 265, loss = 0.04601109
Iteration 266, loss = 0.04442982
Iteration 267, loss = 0.04421355
Iteration 268, loss = 0.04226157
Iteration 269, loss = 0.04363595
Iteration 270, loss = 0.04205349
Iteration 271, loss = 0.04223604
Iteration 272, loss = 0.04031006
Iteration 273, loss = 0.04159741
Iteration 274, loss = 0.04093935
Iteration 275, loss = 0.03951966
Iteration 276, loss = 0.03895989
Iteration 277, loss = 0.03904886
Iteration 278, loss = 0.03805548
Iteration 279, loss = 0.03943738
Iteration 280, loss = 0.03743132
Iteration 281, loss = 0.03819468
Iteration 282, loss = 0.03725501
Iteration 283, loss = 0.03611392
Iteration 284, loss = 0.03751799
Iteration 285, loss = 0.03602859
Iteration 

In [17]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8765957446808511
[[ 51  12]
 [ 17 155]]
              precision    recall  f1-score   support

       False       0.75      0.81      0.78        63
        True       0.93      0.90      0.91       172

   micro avg       0.88      0.88      0.88       235
   macro avg       0.84      0.86      0.85       235
weighted avg       0.88      0.88      0.88       235



# Person 2

In [18]:
df = pd.read_csv("/home/joe/MLP/features_sample_head2_selection.csv")
df = df.astype({'Class_P0': 'bool', 
#                 'head0_AU28': 'bool',
#                 'head0_AU45': 'bool',
#                 'head0_AU26': 'bool',
#                 'head0_AU25': 'bool',
#                 'head0_AU01': 'bool',
#                 'head0_AU05': 'bool',
#                 'head0_AU20': 'bool',
#                 'head0_AU06': 'bool',
#                 'head0_AU23': 'bool',
#                 'head0_AU07': 'bool',
#                 'head0_AU02': 'bool', 
#                 'head0_AU09': 'bool',
#                 'head0_AU17': 'bool',
#                 'head0_AU10': 'bool',
#                 'head0_AU04': 'bool',
#                 'head0_AU12': 'bool',
#                 'head0_AU14': 'bool',
#                 'head0_AU15': 'bool',
                'ClassP2':'bool'})
le = LabelEncoder()
# df['head0_pose'] = le.fit_transform(df['head0_pose'])
# df['head0_pose'] = le.fit_transform(df['head0_pose'])
# df['head2_pose'] = le.fit_transform(df['head2_pose'])
# df['head0_emo'] = le.fit_transform(df['head0_emo'])
# df['head0_emo'] = le.fit_transform(df['head0_emo'])
# df['head2_emo'] = le.fit_transform(df['head2_emo'])

y = df['ClassP2']
x = df.drop(['ClassP2'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.30, random_state=27)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp = imp.fit(x_train)

X_train_imp = imp.transform(x_train)
X_test_imp = imp.transform(x_test)

In [19]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=300, alpha=0.0001, activation='tanh',
                     solver='adam', verbose=10,  random_state=21,tol=0.000000001)

In [20]:
# sfsl = sfs(clf, k_features=10,forward=True)
# print(sfsl.fit(X_test_imp, y_test))
# print(sfsl.subsets_)
# feature_names': ('0', '1', '4', '6', '7', '9', '11', '14', '17', '24')

In [21]:
clf.fit(X_train_imp, y_train)
y_pred = clf.predict(X_test_imp)

Iteration 1, loss = 0.67959590
Iteration 2, loss = 0.61588374
Iteration 3, loss = 0.58874316
Iteration 4, loss = 0.57339714
Iteration 5, loss = 0.56531784
Iteration 6, loss = 0.55652568
Iteration 7, loss = 0.54778030
Iteration 8, loss = 0.54627189
Iteration 9, loss = 0.54461690
Iteration 10, loss = 0.53993356
Iteration 11, loss = 0.53654098
Iteration 12, loss = 0.53021298
Iteration 13, loss = 0.52843148
Iteration 14, loss = 0.52446390
Iteration 15, loss = 0.51960907
Iteration 16, loss = 0.51749485
Iteration 17, loss = 0.51126703
Iteration 18, loss = 0.50646531
Iteration 19, loss = 0.50250207
Iteration 20, loss = 0.49508840
Iteration 21, loss = 0.48858305
Iteration 22, loss = 0.48042729
Iteration 23, loss = 0.47220944
Iteration 24, loss = 0.46222033
Iteration 25, loss = 0.45077801
Iteration 26, loss = 0.43897341
Iteration 27, loss = 0.42587836
Iteration 28, loss = 0.41278359
Iteration 29, loss = 0.39912488
Iteration 30, loss = 0.38481436
Iteration 31, loss = 0.37319714
Iteration 32, los

Iteration 256, loss = 0.10973927
Iteration 257, loss = 0.10940797
Iteration 258, loss = 0.10950328
Iteration 259, loss = 0.10820543
Iteration 260, loss = 0.10850635
Iteration 261, loss = 0.10985594
Iteration 262, loss = 0.10878644
Iteration 263, loss = 0.10908478
Iteration 264, loss = 0.10539557
Iteration 265, loss = 0.10695903
Iteration 266, loss = 0.10574307
Iteration 267, loss = 0.10439510
Iteration 268, loss = 0.10413687
Iteration 269, loss = 0.10293603
Iteration 270, loss = 0.10277483
Iteration 271, loss = 0.10307655
Iteration 272, loss = 0.10140726
Iteration 273, loss = 0.10145571
Iteration 274, loss = 0.10202656
Iteration 275, loss = 0.10148281
Iteration 276, loss = 0.10196156
Iteration 277, loss = 0.10221751
Iteration 278, loss = 0.10005190
Iteration 279, loss = 0.10072133
Iteration 280, loss = 0.09955611
Iteration 281, loss = 0.09783463
Iteration 282, loss = 0.09784285
Iteration 283, loss = 0.09796485
Iteration 284, loss = 0.09836463
Iteration 285, loss = 0.09744745
Iteration 

In [22]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9234042553191489
[[ 70  12]
 [  6 147]]
              precision    recall  f1-score   support

       False       0.92      0.85      0.89        82
        True       0.92      0.96      0.94       153

   micro avg       0.92      0.92      0.92       235
   macro avg       0.92      0.91      0.91       235
weighted avg       0.92      0.92      0.92       235

